In [9]:
import sys
from math import ceil
import pytesseract
import cv2
import numpy as np

print(sys.path)

#CV2 version:  4.8.0
#Numpy version:  1.25.2
#Tesseract version:  0.3.10

print("CV2 version: ", cv2.getVersionString())
print("Numpy version: ", np.__version__)
print("Tesseract version: ", pytesseract.__version__)

#input_video = "./data/vid_sudoku_1.mp4"
#output_folder = "./outputStages/"
#output_basename = "vid_sudoku_1"
#num_frames = 170

input_video = "./data/vid_sudoku_2.mp4"
output_folder = "./outputStages/"
output_basename = "vid_sudoku_2"
num_frames = 185

#####
stage = 0

['c:\\_play\\IDACourse2023\\code\\sudoku_ida\\ocr', 'C:\\Python39\\python39.zip', 'C:\\Python39\\DLLs', 'C:\\Python39\\lib', 'C:\\Python39', '', 'C:\\Users\\Lumi\\AppData\\Roaming\\Python\\Python39\\site-packages', 'C:\\Python39\\lib\\site-packages', 'C:\\Python39\\lib\\site-packages\\win32', 'C:\\Python39\\lib\\site-packages\\win32\\lib', 'C:\\Python39\\lib\\site-packages\\Pythonwin', 'C:\\Python39\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\Lumi\\.ipython']
CV2 version:  4.8.0
Numpy version:  1.25.2
Tesseract version:  0.3.10


In [11]:
# Open video
cap = cv2.VideoCapture(input_video)

frames = []

for _ in range(num_frames):
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(frame)


cap.release()
print("Nb of read frames: ", len(frames))

Nb of read frames:  185


In [12]:
# Try motion compensation
###########################################

def align_images(im1, im2):
    """Align im2 to im1 and return the aligned image"""

    # Convert images to grayscale
    im1_gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
    im2_gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)

    # Define the motion model
    warp_mode = cv2.MOTION_AFFINE

    # Define 2x3 or 3x3 matrices and initialize the matrix to identity
    if warp_mode == cv2.MOTION_HOMOGRAPHY:
        warp_matrix = np.eye(3, 3, dtype=np.float32)
    else:
        warp_matrix = np.eye(2, 3, dtype=np.float32)

    # Specify the number of iterations and termination criteria
    #number_of_iterations = 5000
    #termination_eps = 1e-10
    
    number_of_iterations = 1000
    termination_eps = 1e-4
    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, number_of_iterations, termination_eps)

    # Run the ECC algorithm to estimate the affine transformation
    _, warp_matrix = cv2.findTransformECC(im1_gray, im2_gray, warp_matrix, warp_mode, criteria)

    if warp_mode == cv2.MOTION_HOMOGRAPHY:
        aligned_image = cv2.warpPerspective(im2, warp_matrix, (im1.shape[1], im1.shape[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
    else:
        aligned_image = cv2.warpAffine(im2, warp_matrix, (im1.shape[1], im1.shape[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)

    return aligned_image

########
ref_frame = frames[0]
aligned_frames = [ref_frame]

for idx, fr in enumerate(frames):
    print("Started working on frame ", idx, " ...")
    aligned_frame = align_images(ref_frame, fr)
    aligned_frames.append(aligned_frame)

Started working on frame  0  ...
Started working on frame  1  ...
Started working on frame  2  ...
Started working on frame  3  ...
Started working on frame  4  ...
Started working on frame  5  ...
Started working on frame  6  ...
Started working on frame  7  ...
Started working on frame  8  ...
Started working on frame  9  ...
Started working on frame  10  ...
Started working on frame  11  ...
Started working on frame  12  ...
Started working on frame  13  ...
Started working on frame  14  ...
Started working on frame  15  ...
Started working on frame  16  ...
Started working on frame  17  ...
Started working on frame  18  ...
Started working on frame  19  ...
Started working on frame  20  ...
Started working on frame  21  ...
Started working on frame  22  ...
Started working on frame  23  ...
Started working on frame  24  ...
Started working on frame  25  ...
Started working on frame  26  ...
Started working on frame  27  ...
Started working on frame  28  ...
Started working on frame

In [13]:
# Stage Simple frame averaging and grayscale 
stacked_frames = np.stack(frames, axis=0)


# Compute the average frame
average_frame = np.mean(stacked_frames, axis=0).astype(np.uint8)

# Convert to grayscale
img_gray = cv2.cvtColor(average_frame, cv2.COLOR_BGR2GRAY)

# ----------------
stage = 1
img_stage = img_gray
cv2.imwrite(output_folder + output_basename + "-stage-" + str(stage) +".png", img_stage)
cv2.imwrite('./data/' + output_basename + "-AVG-" + ".png", img_stage)

True